## Multiomics GRN inference evaluation
## VAE-SEM model
### by Jalil Nourisa

In [5]:
!pip install anndata

  Using cached anndata-0.10.9-py3-none-any.whl.metadata (6.9 kB)
  Using cached array_api_compat-1.8-py3-none-any.whl.metadata (1.5 kB)
  Using cached h5py-3.11.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.5 kB)
  Using cached natsort-8.4.0-py3-none-any.whl.metadata (21 kB)
  Using cached pandas-2.2.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached scipy-1.13.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached anndata-0.10.9-py3-none-any.whl (128 kB)
Using cached array_api_compat-1.8-py3-none-any.whl (38 kB)
Using cached h5py-3.11.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.3 MB)
Using cached pandas-2.2.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
Using cached scipy-1.13.1-cp39-cp39-manylinux_2_17_x86_64.manylin

In [4]:
import torch 
# import anndata as ad
import pandas as pd
import numpy as np
import tqdm
# from sklearn.metrics import r2_score, roc_auc_score
# from sklearn.metrics import mean_squared_error 
# from sklearn.decomposition import TruncatedSVD, KernelPCA    
# from sklearn.ensemble import  RandomForestRegressor
import os
import random 
# import category_encoders 
import torch.nn as nn 
# import matplotlib.pyplot as plt

os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

SEED = 0xCAFE
USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    print('using device: cuda')
else:
    print('using device: cpu')
    USE_GPU = False

def seed_everything():
    seed = 42
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TOKENIZERS_PARALLELISM'] = 'true'
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    print('-----Seed Set!-----')
seed_everything()


using device: cuda
-----Seed Set!-----


In [2]:
adata_rna = ad.read_h5ad('../output/scRNA/adata_rna.h5ad')
gene_names = adata_rna.var_names
gene_name_dict = {gene_name: i for i, gene_name in enumerate(gene_names)}
n_genes = len(gene_names)

## Process grn net

In [25]:
# # Download GRN
# # net = pd.read_csv("https://github.com/pablormier/omnipath-static/raw/main/op/collectri-26.09.2023.zip")
# net = pd.read_csv("../output/benchmark/grn_models/scenicplus.csv")

# # Iterate over the edges (regulatory relationships)
# edge_idx = set()
# grn_weights = []
# for gene_a, gene_b, weight in zip(net['source'], net['target'], net['weight']):
#     if (gene_a not in gene_name_dict) or (gene_b not in gene_name_dict):
#         continue  # Consider only gene names that are present in the training data
#     i = gene_name_dict[gene_a]  # Index of first gene
#     j = gene_name_dict[gene_b]  # Index of second gene
#     grn_weights.append(weight)
#     edge_idx.add((i, j))
# edge_idx = np.asarray(list(edge_idx), dtype=int)

# # Convert list of edges into an adjacency matrix
# grn = np.zeros((len(gene_names), len(gene_names)))
# for i, weight in enumerate(grn_weights):
#     weight = 1 if weight>=0 else -1
#     grn[edge_idx[i, 0], edge_idx[i, 1]] = weight

# # Remove rows and columns with no annotation
# grn_mask = np.logical_or(grn.sum(axis=0) > 0, grn.sum(axis=1) > 0)
# grn = grn[grn_mask, :][:, grn_mask]


In [321]:
# grn = pd.read_csv("../output/benchmark/grn_models/scenicplus.csv", index_col=0)
grn = pd.read_csv("https://github.com/pablormier/omnipath-static/raw/main/op/collectri-26.09.2023.zip")
# only keep those with both tf and gene included in gene_names
grn = grn[grn.source.isin(gene_names) & grn.target.isin(gene_names)].reset_index(drop=True)
# binarize
weights = [1 if weight>=0 else -1 for weight in grn.weight  ] 
grn.weight = weights

GT = grn.pivot(index='source', columns='target', values='weight').fillna(0)
 
GT = GT.abs() # binary 
GT.abs().mean().mean()

0.00639388880957349

## Dataset

In [5]:
Y = adata_rna.X.todense() #TODO: modify the train code to deal with sparse
Y

matrix([[0., 0., 0., ..., 0., 0., 2.],
        [0., 1., 2., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 3., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = features 
        self.labels = labels 
    def __len__(self):
        return len(self.features)
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]
features = torch.tensor(Y, dtype=torch.float32) 
labels = torch.tensor(Y, dtype=torch.float32)

dataset = Dataset(features=features, labels=labels)

### Encode data

In [7]:
# class MultiOutputTargetEncoder:
#     def __init__(self):
#         self.encoders: List[category_encoders.leave_one_out.LeaveOneOutEncoder] = []
        
#     @staticmethod
#     def new_encoder() -> category_encoders.leave_one_out.LeaveOneOutEncoder:
#         return category_encoders.leave_one_out.LeaveOneOutEncoder(return_df=False)
    
#     def fit(self, X: np.ndarray, y: np.ndarray) -> None:
#         self.encoders = []
#         for j in tqdm.tqdm(range(y.shape[1]), desc='fit LOO encoders'):
#             self.encoders.append(MultiOutputTargetEncoder.new_encoder())
#             self.encoders[-1].fit(X, y[:, j])
    
#     def transform(self, X: np.ndarray) -> np.ndarray:
#         Z = []
#         for encoder in tqdm.tqdm(self.encoders, desc='transform LOO encoders'):
#             y_hat = encoder.transform(X)
#             Z.append(y_hat)
#         Z = np.asarray(Z)
#         return np.transpose(Z, (1, 0, 2))
# encoder = MultiOutputTargetEncoder()

# encoder.fit(np.asarray([df_train.index.get_level_values(var) for var in features_X]).T, df_train.values)

# X = encoder.transform(np.asarray([df_train.index.get_level_values(var) for var in features_X]).T)
# X_submit = encoder.transform(np.asarray([df_test.index.get_level_values(var) for var in features_X]).T)

### NN design

In [8]:
class NN_AE(torch.nn.Module): 
    '''
    Simple AE design.
    '''
    def __init__(self, n_genes:int,n_nodes_hidden:int=120):
        torch.nn.Module.__init__(self)
        self.n_genes = n_genes
        dropout_rate = .1
        
        self.mlp = nn.Sequential(
                nn.Linear(n_genes, n_nodes_hidden),
                nn.LeakyReLU(0.2),
                nn.Dropout(dropout_rate),  
                
                nn.Linear(n_nodes_hidden, 64),
                nn.LeakyReLU(0.2),
                nn.Dropout(dropout_rate),  
            
                nn.Linear(64, 120),
                nn.LeakyReLU(0.2),
                nn.Dropout(dropout_rate),  
            
                nn.Linear(n_nodes_hidden, n_genes)
            )

    def forward(self, x: torch.Tensor):
        
        x = self.mlp(x)
        
        return x, None, None


In [9]:
# def background_noise( #we use dropouts instead of this for pseudobulked data
#     *size: int,
#     cutoff: float = 0.05,
#     device: str = 'cuda',
#     generator: torch.Generator = None) -> torch.Tensor:
#     sign = 2 * torch.randint(0, 2, size, device=device) - 1

#     return sign * torch.log10(cutoff +  torch.rand(*size, generator=generator, device=device) * (1. - cutoff))
# class Scaler(torch.nn.Module): #from antoine
    
#     def __init__(self, m: int) -> None:
#         torch.nn.Module.__init__(self)
#         self.m: int = m
#         self.a: torch.Tensor = torch.nn.Parameter(torch.ones((1, self.m)))
#         self.b: torch.Tensor = torch.nn.Parameter(torch.zeros((1, self.m)))
    
#     def forward(self, X: torch.Tensor) -> torch.Tensor:
#         return self.a * X + self.b


###  Train

In [10]:
from torch.utils.data import DataLoader
def train_nn(model, dataloader, n_epoch, loss_func):
    model = model.to('cuda')
    model.train()

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, eps=1e-8)
    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.9, 
                patience=5, threshold_mode='rel', threshold=0.0001, cooldown=5, min_lr=1e-10, eps=1e-8)

    pbar = tqdm.tqdm(range(n_epoch))
    for i_epoch in pbar:
        rel_loss_store = []
        Y_pred_stack = []
        Y_true_stack = []
        for batch_idx, (feature_batch, label_batch) in enumerate(dataloader):
            
            optimizer.zero_grad()
            data_batch = feature_batch.to('cuda') #TODO: this should be done once 
            label_batch = label_batch.to('cuda')
            
            if False: # add background noise
                generator = torch.Generator(device='cuda').manual_seed(32)
                label_batch = label_batch + 0.2 * background_noise(*label_batch.size(), generator=generator) #TODO: optimize the weight

            x_pred, mu, log_var = model(data_batch) #forward

            Y_true_stack.append(label_batch.cpu().detach().numpy())
            Y_pred_stack.append(x_pred.cpu().detach().numpy())

            loss_x = loss_func(x_pred, label_batch)

            #loss_KL =  - 0.5 * torch.sum(1.0 + log_var - mu.pow(2) - log_var.exp()) #TODO: fix this

            beta = 1
            #loss = loss_x + beta*loss_KL
            loss = loss_x 
            loss.backward()
            optimizer.step()

            # baseline pred
            Y_pred_mean = torch.mean(label_batch, axis=0)
            loss_baseline = loss_func(label_batch, Y_pred_mean)
            rel_loss = loss_x/loss_baseline
            rel_loss_store.append(rel_loss.item())
        # if i_epoch%10==0:
        #     # AUROC
        #     mask = ~np.eye(n_genes, dtype=bool)
        #     grn_pred = np.abs(model.A.cpu().data.numpy())
        #     print('AUROC', roc_auc_score(np.abs(grn_net[mask]), grn_pred[mask]))

        mean_rel_loss = np.mean(rel_loss_store)
        scheduler.step(mean_rel_loss)

        y_pred = np.concatenate(Y_pred_stack, axis=0)
        y_true = np.concatenate(Y_true_stack, axis=0)

        r2 = r2_score(y_true, y_pred, multioutput='variance_weighted')

        pbar.set_description(f'Rel loss: {mean_rel_loss:.3f}, R2:{r2:.3f}')
    

    return model

batch_size = int(Y.shape[0]/20)
n_epoch = 10
n_model = 1
num_workers = 4
shuffle_data = True
loss_func = lambda y_pred, y_true: torch.sum(torch.square(y_pred-y_true))

models = []
# y_pred_submit_all = []
for i in range(n_model):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle_data, num_workers=num_workers)
    model = NN_AE(n_genes)
    model = train_nn(model, dataloader, n_epoch, loss_func)
    
    models.append(model)

Rel loss: 0.425, R2:0.574: 100%|██████████| 10/10 [01:06<00:00,  6.67s/it]


## Extract GRN

In [11]:
# Register hooks to capture activations for all layers
nodel = model.to('cpu')
model.eval()
layers = model.mlp

activations = []

def get_activation_hook(layer_index):
    def hook(module, input, output):
        activations[layer_index] = output.detach()
    return hook

# Initialize activations list
activations = [None] * len(layers)

# Register hooks for all Linear layers
for idx, layer in enumerate(model.mlp):
    layer.register_forward_hook(get_activation_hook(idx))

# Forward pass to get activations
with torch.no_grad():
    _ = model(features)


In [12]:
print(activations[0][0][0:5])
print(activations[1][0][0:5])
print(activations[2][0][0:5])

tensor([-14.6386, -11.2354,  -2.3267,   3.0302,  -0.5415])
tensor([-2.9277, -2.2471, -0.4653,  3.0302, -0.1083])
tensor([-2.9277, -2.2471, -0.4653,  3.0302, -0.1083])


In [103]:
# adjusted the weights based on the activaition degrees (due to activaition function)
weights_adj_all = []
n_layers = len(model.mlp)
for i in range(n_layers):
    layer = model.mlp[i]
    if isinstance(layer, nn.Linear):
        weight = layer.weight.detach().numpy().T
        if i < (n_layers-1):
            # weights_adj =  weight 
            activation_per_node = activations[i+1].mean(axis=0, keepdims=True).numpy() #activation function is always one ahead
            weights_adj =  weight * activation_per_node
        else:
            weights_adj = weight
        weights_adj_all.append(weights_adj)
        print(weights_adj.shape)

(22778, 120)
(120, 64)
(64, 120)
(120, 22778)


In [444]:
# calculate regulatory interactions
net = weights_adj_all[0]
for weights in weights_adj_all[1:]:
    net = net @ weights
net_df = pd.DataFrame(net, columns=gene_names, index=gene_names)

In [448]:
# net_df = pd.read_csv("../output/benchmark/grn_models/scenicplus.csv")
# net_df = net_df.pivot(index='source', columns='target', values='weight').fillna(0)
# net_df = net_df.abs()
# net_df


## Evaluate GRN

In [452]:
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, f1_score, matthews_corrcoef, average_precision_score

def metric_curve(y_true, y_scores):
    tt = np.linspace(0, y_scores.max(), 20)
    matt_scores = []
    f1_scores = []
    # print(tt)
    for t in tt[:-1]:
        y_pred = y_scores>t
        f1_scores.append(f1_score(y_true, y_pred))
        matt_scores.append(matthews_corrcoef(y_true, y_pred))
    
    return  matt_scores, f1_scores

for top_n in [5, 10, 100]:
    # choose the top n genes with highest regulators
    n_regulators = GT.sum(axis=0)
    # GT_subset = GT[n_regulators.nlargest(top_n).index]
    GT_subset = GT.copy()

    # only those tfs and targets given in GT
    def match_dfs(df1, df2):
        df1 = df1.loc[df1.index.isin(df2.index), df1.columns.isin(df2.columns)]
        df2 = df2.loc[df2.index.isin(df1.index), df2.columns.isin(df1.columns)]
        return df1, df2
    net_subset, GT_subset = match_dfs(net_df, GT_subset)
    # reindex for compatibility
    net_subset = net_subset.reindex(index=GT_subset.index, columns=GT_subset.columns)

    y_pred = net_subset.values.flatten()
    y_true = GT_subset.values.flatten()

    # no negative reg
    y_pred = np.abs(y_pred)

    print('n links:',len(y_pred))

    assert np.isnan(y_pred).any()==False
    assert y_true.shape==y_pred.shape

    # metrics
    aucroc = roc_auc_score(y_true, y_pred)
    # precision, recall, tt = precision_recall_curve(y_true, y_pred)
    # aucpr = (precision * recall).mean()
    precision_mean = average_precision_score(y_true, y_pred)

    matt_scores, f1_scores = metric_curve(y_true, y_pred)
    matt_mean = np.mean(matt_scores)
    f1_mean = np.mean(f1_scores)

    print(f'{top_n=}, {aucroc=}, {precision_mean=}, {matt_mean=}, {f1_mean=}')

n links: 4370736
top_n=5, aucroc=0.5082225495511123, precision_mean=0.006695495302780784, matt_mean=0.0005570626029492135, f1_mean=0.001858002846117856
n links: 4370736
top_n=10, aucroc=0.5082225495511123, precision_mean=0.006695495302780784, matt_mean=0.0005570626029492135, f1_mean=0.001858002846117856
n links: 4370736


KeyboardInterrupt: 

n links: 303
top_n=5, aucroc=0.5264077669902912, precision_mean=0.3669703794235179, matt_mean=0.10818906873658679, f1_mean=0.08888404098938463
n links: 707
top_n=10, aucroc=0.5187080867850099,  precision_mean=0.3046555354494385, matt_mean=0.09383949611263599, f1_mean=0.07612230579002001
n links: 7272
top_n=100, aucroc=0.5232494638773373, precision_mean=0.1546841109780841, matt_mean=0.059091840774979656, f1_mean=0.036139223144212335
n links: 74235
top_n=1000, aucroc=0.532563048262141, precision_mean=0.05838751393542791, matt_mean=0.06267851369070392, f1_mean=0.042948548762372384
n links: 370670
top_n=5000, aucroc=0.5308883277329034, precision_mean=0.021350126982468423, matt_mean=0.045122637323227084, f1_mean=0.028030001732845523


In [109]:
aa

NameError: name 'aa' is not defined

In [ ]:
from local_utils import plots
plots.plot_cumulative_density(reg_net.flatten())


## Saliency map approach

In [ ]:
# !pip install captum

In [ ]:
from captum.attr import IntegratedGradients

/root/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Initialize NN explainer
# baseline = torch.zeros((1, X.size()[1], X.size()[2]))
# if USE_GPU:
#     baseline = baseline.cuda()
ig = IntegratedGradients(model)

# Sort the columns of the goldstandard network based on the coverage of annotations
idx = np.argsort(np.sum(grn, axis=0))[::-1]

all_z_target, all_z_pred = [], []
for k in range(5):
    
    # Select column `j` from the goldstandard GRN
    j = int(idx[k])
    z_target = grn[:, j]
    z_target[j] = np.nan  # Assume that a gene cannot regulate itself -> will be filtered out later
    all_z_target.append(z_target)
    
    # Compute feature attribution vectors for many input points and average over the results
    z_pred = np.zeros(len(gene_names))
    for i in tqdm.tqdm(range(1000)):
        # Generate random input based on the null distribution
        input_ = background_noise(X.size()[1], X.size()[2], device=baseline.device)
        
        # Compute feature attribution
        attributions = ig.attribute(input_, baseline, target=j, return_convergence_delta=False)
        # An alternative is to use input samples from the training set:
        # attributions = ig.attribute(X[i, ...], baseline, target=j, return_convergence_delta=False)
        # By averaging across all samples from the same cell type, one can build a biological network
        # that is cell type-specific.
        
        z_pred += np.mean(np.abs(np.squeeze(attributions.cpu().data.numpy())), axis=1)
    all_z_pred.append(z_pred[grn_mask])
    
# Compute performance metrics
all_z_target = np.concatenate(all_z_target, axis=0)
all_z_pred = np.concatenate(all_z_pred, axis=0)
mask = ~np.isnan(all_z_target)  # Discard self-regulations
print(f'AUROC: {roc_auc_score(all_z_target[mask], all_z_pred[mask])}')
print(f'AUPR : {average_precision_score(all_z_target[mask], all_z_pred[mask])}')

In [ ]:
 # predict
# model.eval()
# y_pred_submit, mu, log_var = model(torch.tensor(X_submit, dtype=torch.float32, device='cuda'))
# y_pred_submit = y_pred_submit.cpu().detach().numpy()
# y_pred_submit_all.append(y_pred_submit)

# print('r2:', r2_score(df_test, y_pred_submit, multioutput='variance_weighted'))
# models.append(model)

In [ ]:
model.A

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]], device='cuda:0')

## Predict

In [ ]:
print('number of models:', len(y_pred_submit_all))
y_pred_submit_mean = np.mean(y_pred_submit_all, axis=0)
print('r2:', r2_score(df_test, y_pred_submit_mean, multioutput='variance_weighted'))
print('mse:', mean_squared_error(y_pred_submit_mean, df_test.values))
print('mrrmse:', mrrmse(y_pred_submit_mean, df_test.values))

In [ ]:
r2: 0.24227989212421114
mse: 1.7873100056454099
mrrmse: 0.8062594076816958

## Baseline RF

In [ ]:
def encode(df_train, df_test, feature_space):
    # encode each factor
    x_encoded_dict = {}
    for feature in feature_space:
        index = df_train.index.get_level_values(feature)
        n_com = min([len(index.unique()), 50])
        # var_x = TruncatedSVD(n_components=n_com, n_iter=12, random_state=random_state).fit_transform(df_train)
        var_x = KernelPCA(n_components=n_com, kernel='linear', random_state=random_state).fit_transform(df_train)
        x_encoded = pd.DataFrame(var_x, index=index).reset_index()
        x_encoded = x_encoded.groupby(feature).mean()
        x_encoded_dict[feature] = x_encoded
    # create X and X_submit
    X = []
    X_submit = []
    for i_feature, feature in enumerate(feature_space):
        # encode train data
        index = df_train.index.get_level_values(feature)
        feature_encoded = np.asarray([x_encoded_dict[feature].loc[name].values for name in index])
        if i_feature == 0:
            X = feature_encoded
        else: 
            X = np.concatenate([X, feature_encoded], axis=1)
        
        # encode test data
        index = df_test.index.get_level_values(feature)
        feature_encoded = np.asarray([x_encoded_dict[feature].loc[name].values for name in index])
        if i_feature == 0:
            X_submit = feature_encoded
        else: 
            X_submit = np.concatenate([X_submit, feature_encoded], axis=1)
    return X, X_submit


random_state = 32
n_components = 50
X_rf, X_submit_rf = encode(df_train, df_test, features_X)
emb_model = RandomForestRegressor(n_estimators=100, random_state=random_state)
reducer = TruncatedSVD(n_components=n_components, n_iter=12, random_state=random_state)
Y = reducer.fit_transform(df_train)

emb_model.fit(X_rf, Y)
y_pred_submit = reducer.inverse_transform(emb_model.predict(X_submit_rf))

print('r2:', r2_score(df_test, y_pred_submit, multioutput='variance_weighted'))
print('mse:', mean_squared_error(y_pred_submit, df_test.values))
print('mrrmse:', mrrmse(y_pred_submit, df_test.values))

In [ ]:
r2: 0.24227989212421114
mse: 1.7873100056454099
mrrmse: 0.8062594076816958